In [ ]:
from soldb import main, parse_arguments
import os

# Define Variables
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

username = 'tmind'
param_eval = 'tmind'
mode = 'update'

arguments = ["--username" , username,
              "--eval" , param_eval,
              #"--filter", "C~Biologist" 
                  #"--type", "fuseddeck"  
]

# Now you can use the imported functions
args = parse_arguments(arguments)

# Example usage:

        # F =>  String: Faction                  'Alloyn'              F=Alloyn
        # D =>  String: Deckname                 'Insane'              D~Forge
        # FB => String: Forgeborn               'Ironbeard'            FB=Ironbeard
        # C  => List:   Cardnames               'Digitize'             C~Digitize
        # A  => List:   Forgeborn - Ability     'Army Commander'       A~'Army Commander'
        # K  => Dict:   Composition             'Robot'                K:Robot > 3


In [1]:
from pyvis.network import Network

def graphToNet(graph):
    net = Network(notebook=True, directed=True, height="1500px", width="2000px", cdn_resources='in_line')
    net.force_atlas_2based()
    net.from_nx(graph)
    # Display the network
    net.show_buttons(filter_=['physics'])
    print("Displaying Graph!")
    #display(net.show('graph.html'))
    return net

In [7]:
import ipywidgets as widgets
from IPython.display import display, HTML, IFrame

from MongoDB.DatabaseManager import DatabaseManager
import GlobalVariables

from networkx import descendants
from pyvis.network import Network
from CardLibrary import Deck

GlobalVariables.username = 'tmind'
myDB = DatabaseManager(GlobalVariables.username)

factionNames = ['Alloyin', 'Nekrium', 'Tempys', 'Uterra']
deckNames = []

out = widgets.Output()

factionToggleA = widgets.ToggleButtons(
    options=factionNames,
    description='Faction:',
    disabled=False,
    button_style='info',  # Remove the initial button_style
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
) 

def update_button_style(change):
    if change['new'] == 'Alloyin':
        factionToggleA.button_style = 'info'
    elif change['new'] == 'Nekrium':
        factionToggleA.button_style = 'warning'
    elif change['new'] == 'Tempys':
        factionToggleA.button_style = 'danger'
    elif change['new'] == 'Uterra':
        factionToggleA.button_style = 'success'

factionToggleA.observe(update_button_style, 'value')

dropdown = widgets.Dropdown()
def button_on_click(button):
    myDeck = Deck.lookup(dropdown.value)
    myGraph = myDeck.create_graph_children()
    net = graphToNet(myGraph.G)
    with out:
        out.clear_output() 
        display(net.show(f"{dropdown.value}.html"))
    
        
button = widgets.Button(description="Show Graph")
button.on_click(button_on_click)

def update_items(*args):
    
    deckCursor = myDB.find('Deck', {'faction' : factionToggleA.value})
    deckNames = [deck['name'] for deck in deckCursor]
    dropdown.options = deckNames

# Call update items once to set the initial items 
update_items()

factionToggleA.observe(update_items, 'value')


display(factionToggleA, dropdown, button)
display(out)




ToggleButtons(button_style='success', description='Faction:', options=('Alloyin', 'Nekrium', 'Tempys', 'Uterra…

Dropdown(options=('The Square Proposing Sentry Angels', 'The Raiders of Leading Capitals', 'The Common Wrench …

Button(description='Show Graph', style=ButtonStyle())

Output()